# Train a gesture recognition model for microcontroller use

This notebook demonstrates how to train a 20kb gesture recognition model for [TensorFlow Lite for Microcontrollers](https://tensorflow.org/lite/microcontrollers/overview). It will produce the same model used in the [motion_detector](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/micro/examples/motion_detector) example application.

The model is designed to be used with [Google Colaboratory](https://colab.research.google.com).

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/motion_detector/train/train_motion_detector_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/motion_detector/train/train_motion_detector_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


Training is much faster using GPU acceleration. Before you proceed, ensure you are using a GPU runtime by going to **Runtime -> Change runtime type** and selecting **GPU**. Training will take around 5 minutes on a GPU runtime.

## Configure dependencies

Run the following cell to ensure the correct version of TensorFlow is used.

We'll also clone the TensorFlow repository, which contains the training scripts, and copy them into our workspace.

In [ ]:
# Clone the repository from GitHub
!git clone --depth 1 -q https://github.com/tensorflow/tensorflow
# Copy the training scripts into our workspace
!cp -r tensorflow/tensorflow/lite/micro/examples/motion_detector/train train

## Prepare the data

Next, we'll download the data and extract it into the expected location within the training scripts' directory.

In [ ]:
# Download the data we will use to train the model
!wget http://download.tensorflow.org/models/tflite/motion_detector/data.tar.gz
# Extract the data into the train directory
!tar xvzf data.tar.gz -C train 1>/dev/null

We'll then run the scripts that split the data into training, validation, and test sets.

In [ ]:
# The scripts must be run from within the train directory
%cd train
# Prepare the data
!python data_prepare.py
# Split the data by person
!python data_split_person.py

## Load TensorBoard

Now, we set up TensorBoard so that we can graph our accuracy and loss as training proceeds.

In [ ]:
# Load TensorBoard
%load_ext tensorboard
%tensorboard --logdir logs/scalars

## Begin training

The following cell will begin the training process. Training will take around 5 minutes on a GPU runtime. You'll see the metrics in TensorBoard after a few epochs.

In [2]:
!python train.py --model CNN --person true

Start to load data...
train_data_length:14025
valid_data_length:124


2023-07-18 09:29:29.512769: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2023-07-18 09:29:29.512942: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2023-07-18 09:29:29.547993: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1614] Profiler found 1 GPUs


test_data_length:164
Start to build net...
Built CNN.
Start training...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 3, 8)         104       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 42, 1, 8)          0         
_________________________________________________________________
dropout (Dropout)            (None, 42, 1, 8)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 42, 1, 16)         528       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 1, 16)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 1, 16)         0         
__________________________________________________

2023-07-18 09:29:29.551677: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cupti64_112.dll'; dlerror: cupti64_112.dll not found
2023-07-18 09:29:29.555015: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cupti.dll'; dlerror: cupti.dll not found
2023-07-18 09:29:29.555314: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1666] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2023-07-18 09:29:29.555701: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2023-07-18 09:29:29.555872: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1757] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.
2023-07-18 09:29:32.515286: I tensorflow/core/platform/cpu_feature_guard.cc:142] This Tenso

1000/1000 [==============================] - 5s 5ms/step - loss: 1.1316 - accuracy: 0.4680 - val_loss: 0.6904 - val_accuracy: 0.6774
Epoch 3/50

1000/1000 [==============================] - 5s 5ms/step - loss: 0.8860 - accuracy: 0.5174 - val_loss: 0.5585 - val_accuracy: 0.7137
Epoch 4/50

1000/1000 [==============================] - 5s 5ms/step - loss: 0.8117 - accuracy: 0.6151 - val_loss: 0.4383 - val_accuracy: 0.7823
Epoch 5/50

1000/1000 [==============================] - 5s 5ms/step - loss: 0.7657 - accuracy: 0.6374 - val_loss: 0.6957 - val_accuracy: 0.6169
Epoch 6/50

1000/1000 [==============================] - 5s 5ms/step - loss: 0.7006 - accuracy: 0.6516 - val_loss: 0.5231 - val_accuracy: 0.7218
Epoch 7/50

1000/1000 [==============================] - 5s 5ms/step - loss: 0.6953 - accuracy: 0.6639 - val_loss: 0.3497 - val_accuracy: 0.8105
Epoch 8/50

1000/1000 [==============================] - 5s 5ms/step - loss: 0.6458 - accuracy: 0.6694 - val_loss: 0.3495 - val_accuracy: 0.80

## Create a C source file

The `train.py` script writes a model, `model.tflite`, to the training scripts' directory.

In the following cell, we convert this model into a C++ source file we can use with TensorFlow Lite for Microcontrollers.

In [ ]:
# Install xxd if it is not available
!apt-get -qq install xxd
# Save the file as a C source file
!xxd -i model.tflite > /content/model.cc
# Print the source file
!cat /content/model.cc